In [ ]:
import os
import sqlite3
import time
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
import warnings

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from transformers.data.data_collator import DataCollatorWithPadding
from transformers.trainer_callback import EarlyStoppingCallback
from transformers.training_args import TrainingArguments
from transformers.trainer import Trainer

from datasets import Dataset
from tqdm.auto import tqdm

# === Silence warnings/logging ===
warnings.filterwarnings("ignore", message=".*pin_memory.*")
logging.getLogger("transformers").setLevel(logging.ERROR)

# === Model list ===
MODELS = [
    "distilroberta-base",
    "distilbert-base-uncased",
    "nreimers/MiniLM-L6-H384-uncased",
    "google/electra-small-discriminator",
    "huawei-noah/TinyBERT_General_4L_312D"
]

# === Dataset info ===
DATASETS = {
    "news": "data/news.db",
    "reddit": "data/reddit_discussions.db",
    "twitter": "data/twitter.db"
}

# === Metric computation ===
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, zero_division=0),
        "precision": precision_score(labels, preds, zero_division=0),
        "recall": recall_score(labels, preds)
    }

# === Benchmark loop over datasets ===
all_results = []

for source_name, db_path in tqdm(DATASETS.items(), desc="🔬 Benchmarking Datasets"):
    print(f"\n📂 Benchmarking on dataset: {source_name}")
    con = sqlite3.connect(db_path)
    df = pd.read_sql("SELECT summary AS text, label FROM labeled_data", con)
    con.close()
    df.dropna(subset=["text", "label"], inplace=True)
    df["label"] = df["label"].astype(int)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for model_name in tqdm(MODELS, desc=f"🔬 Benchmarking Models on {source_name}"):
        print(f"\n🔍 Model: {model_name}")
        fold_scores, all_probs, all_labels = [], [], []

        for fold, (train_idx, val_idx) in enumerate(skf.split(df["text"], df["label"])):
            print(f" ↪ Fold {fold+1}/5")
            train_df, val_df = df.iloc[train_idx], df.iloc[val_idx]

            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
            safe_model_name = model_name.replace("/", "_").replace("-", "_")
            model_dir = f"./results/{source_name}_{safe_model_name}_fold{fold+1}"

            if os.path.exists(model_dir):
                model = AutoModelForSequenceClassification.from_pretrained(model_dir)
            else:
                model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

            train_ds = Dataset.from_pandas(train_df).map(lambda x: tokenizer(x["text"], truncation=True, max_length=512), batched=True)
            val_ds = Dataset.from_pandas(val_df).map(lambda x: tokenizer(x["text"], truncation=True, max_length=512), batched=True)

            train_ds = train_ds.rename_column("label", "labels")
            val_ds = val_ds.rename_column("label", "labels")
            train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
            val_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

            args = TrainingArguments(
                output_dir=model_dir,
                per_device_train_batch_size=8,
                per_device_eval_batch_size=16,
                num_train_epochs=5,
                eval_strategy="epoch",
                save_strategy="epoch",
                save_total_limit=1,
                load_best_model_at_end=True,
                logging_steps=50,
                report_to="none",
                disable_tqdm=True,
                fp16=False,
                learning_rate=2e-5,
                no_cuda=True,
            )

            checkpoint_path = os.path.join(model_dir, "pytorch_model.bin")
            if os.path.exists(checkpoint_path):
                model = AutoModelForSequenceClassification.from_pretrained(model_dir)
                model.to("cpu")
                train_time = 0.0
            else:
                trainer = Trainer(
                    model=model,
                    args=args,
                    train_dataset=train_ds,
                    eval_dataset=val_ds,
                    compute_metrics=compute_metrics,
                    data_collator=DataCollatorWithPadding(tokenizer),
                    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
                )
                start = time.time()
                trainer.train()
                trainer.save_model(model_dir)
                train_time = round(time.time() - start, 2)

            trainer = Trainer(
                model=model,
                args=args,
                train_dataset=train_ds,
                eval_dataset=val_ds,
                compute_metrics=compute_metrics,
                data_collator=DataCollatorWithPadding(tokenizer),
                callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
            )

            metrics = trainer.evaluate()
            metrics["training_time"] = train_time

            preds = trainer.predict(val_ds)
            probs = torch.nn.functional.softmax(torch.tensor(preds.predictions), dim=1).numpy()
            true_labels = preds.label_ids

            all_probs.append(probs[:, 1])
            all_labels.append(true_labels)

            fpr, tpr, _ = roc_curve(true_labels, probs[:, 1])
            roc_auc = auc(fpr, tpr)
            metrics["auc"] = roc_auc

            os.makedirs("roc_curves", exist_ok=True)
            plt.figure()
            plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"AUC={roc_auc:.2f}")
            plt.plot([0, 1], [0, 1], linestyle='--', color='navy')
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title(f"{model_name} Fold {fold+1} ROC ({source_name})")
            plt.legend(loc="lower right")
            plt.savefig(f"roc_curves/{source_name}_{safe_model_name}_fold{fold+1}.png")
            plt.close()

            fold_scores.append({k.replace("eval_", ""): v for k, v in metrics.items()})

        # Overall ROC curve
        if fold_scores:
            all_probs_flat = np.concatenate(all_probs)
            all_labels_flat = np.concatenate(all_labels)
            fpr, tpr, _ = roc_curve(all_labels_flat, all_probs_flat)
            overall_auc = auc(fpr, tpr)

            plt.figure()
            plt.plot(fpr, tpr, color='green', lw=2, label=f"AUC={overall_auc:.2f}")
            plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title(f"{model_name} Overall ROC ({source_name})")
            plt.legend(loc="lower right")
            os.makedirs("roc_overall", exist_ok=True)
            plt.savefig(f"roc_overall/{source_name}_{safe_model_name}_overall.png")
            plt.close()

            avg = pd.DataFrame(fold_scores).mean().to_dict()
            avg.update({"model": model_name, "auc": overall_auc, "source": source_name})
            all_results.append(avg)

# === Save Final Results ===
results_df = pd.DataFrame(all_results)
required_cols = ["source", "model", "accuracy", "f1", "precision", "recall", "auc", "training_time"]
results_df = results_df[[col for col in required_cols if col in results_df.columns]]
results_df.to_csv("multimodal_results.csv", index=False)
print("\n✅ Multimodal Benchmarking complete. Results saved to multimodal_results.csv")


🔬 Benchmarking Datasets:   0%|          | 0/3 [00:00<?, ?it/s]


📂 Benchmarking on dataset: news


🔬 Benchmarking Models on news:   0%|          | 0/5 [00:00<?, ?it/s]


🔍 Model: distilroberta-base
 ↪ Fold 1/5


/Users/gchhetri/Developer/Python/15min-city-bench/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


...

{'eval_loss': 0.6873769164085388, 'eval_accuracy': 0.5, 'eval_f1': 0.6666666666666666, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_runtime': 0.015, 'eval_samples_per_second': 266.453, 'eval_steps_per_second': 66.613, 'epoch': 4.0}


/Users/gchhetri/Developer/Python/15min-city-bench/.venv/lib/python3.12/site-packages/transformers/trainer.py:4090: UserWarning: mtime may not be reliable on this filesystem, falling back to numerical ordering
  warnings.warn("mtime may not be reliable on this filesystem, falling back to numerical ordering")


{'eval_loss': 0.687095046043396, 'eval_accuracy': 0.5, 'eval_f1': 0.6666666666666666, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_runtime': 0.0146, 'eval_samples_per_second': 273.597, 'eval_steps_per_second': 68.399, 'epoch': 5.0}


/Users/gchhetri/Developer/Python/15min-city-bench/.venv/lib/python3.12/site-packages/transformers/trainer.py:4090: UserWarning: mtime may not be reliable on this filesystem, falling back to numerical ordering
  warnings.warn("mtime may not be reliable on this filesystem, falling back to numerical ordering")


{'train_runtime': 2.1065, 'train_samples_per_second': 37.978, 'train_steps_per_second': 4.747, 'train_loss': 0.679874038696289, 'epoch': 5.0}
{'eval_loss': 0.687095046043396, 'eval_model_preparation_time': 0.0015, 'eval_accuracy': 0.5, 'eval_f1': 0.6666666666666666, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_runtime': 0.0175, 'eval_samples_per_second': 229.056, 'eval_steps_per_second': 57.264}

✅ Multimodal Benchmarking complete. Results saved to multimodal_results.csv
